## Scrape imdb website to find the top 250 movies

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

def scrape_imdb_top_movies(url):
    #The resulting parsed HTML is stored in the variable "soup", which can then be used to extract information from the webpage using various BeautifulSoup methods and functions.
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Start from the ipc-page-grid class
    page_grid = soup.find('div', class_='ipc-page-grid')

    # Find the main content column
    main_column = page_grid.find('div', class_='ipc-page-grid__item--span-2')

    # Find all movie items within the main column
    movie_items = main_column.find_all('li', class_='ipc-metadata-list-summary-item') if main_column else []

    movies = []

    print(movie_items.count)

    for item in movie_items:
        title_elem = item.find('h3', class_='ipc-title__text')
        title = title_elem.text
        movies.append(title)

    return pd.DataFrame(movies, columns=['Title'])


url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
driver = webdriver.Firefox()
driver.get(url)
time.sleep(3)

movie_df = scrape_imdb_top_movies(url)
movie_df.head()
driver.quit()

<built-in method count of ResultSet object at 0x000001ED8C3C5400>


,Title
0,1. The Shawshank Redemption
1,2. The Godfather
2,3. The Dark Knight
3,4. The Godfather Part II
4,5. 12 Angry Men


### Generate the CSV based on the scrapped results

In [5]:
import os

filename = 'movies_scraped.csv'

outdir = '../data'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, filename)    
movie_df.to_csv(fullname)